In [36]:
from unittest import result
import pandas as pd
from vakantiedagen import get_vakantiedagen
from moneyed import Money, EUR


# debugger
def debug(print_statement):
    print(print_statement)


"""_summary_
Neemt cursusinformatie en geeft een dataframe waarin de omzet lineair per dag verdeeld
wordt. Houdt daarbij rekening met gegeven jaaromzetten. Als een jaaromzet gegeven is,
wordt die omzet binnen het jaar lineair verdeeld.
"""


def calc_omzet_per_dag(row) -> pd.DataFrame:
    assert "Cursuscode" in row.index
    assert "Startdatum" in row.index
    assert "Einddatum" in row.index
    assert "Omzet" in row.index

    debug("############")
    debug(f"Start berekening voor {row['Cursuscode']}")

    # dit zijn alle werkdagen in totaal (als datum datatype) inclusief mogelijke vakantiedagen.
    # NB DIT ZIJN WERKDAGEN ('freq = "B"')! Er wordt geen omzet genomen in het weekend.
    daterange = pd.date_range(
        row["Startdatum"], row["Einddatum"], freq="B", inclusive="both", unit="s"
    )

    # Vervolgens wordt een lijst gemaakt van alle vakantiedagen.
    # Bron = API Rijksoverheid
    vakantiedagen = get_vakantiedagen()
    debug(
        f"# get_vakantiedagen() debuginfo\nEerste vakantiedag: {vakantiedagen[0]}\nLaatste vakantiedag: {vakantiedagen[-1]}"
    )
    assert vakantiedagen[0] < daterange[0].date()
    assert vakantiedagen[-1] > daterange[-1].date()
    debug(
        "Periode tussen start en einde cursus wordt volledig gedekt door definities vakantieperiodes."
    )

    lesdagen_per_jaar = {}

    # Nu maken we een hashmap (dictionary) met sleutels voor elk jaar dat voorkomt in de
    # handmatige definities van jaaromzetten.
    # De overige dagen worden onder de sleutel 'resterend' opgenomen.
    # Dagen die in een vakantieperiode vallen worden overgeslagen.
    for datum in daterange:
        if datum.date() in vakantiedagen:
            continue
        sleutel = "resterend"
        if datum.year in row.index:
            sleutel = datum.year
        if sleutel in lesdagen_per_jaar:
            lesdagen_per_jaar[sleutel].append(datum)
        else:
            lesdagen_per_jaar[sleutel] = [datum]

    df_result = pd.DataFrame()

    # We gaan per jaar de omzet per lesdag berekenen.
    # Als voor een jaar een omzetbedrag gegeven is, wordt die omzet lineair
    # verdeeld over de werkdagen van dat jaar
    restomzet = Money(row["Omzet"], EUR)  # de totaal resterende, te verdelen omzet
    debug(f"# Restomzet bij aanvang van de berekening is {restomzet}")
    for jaar, dagen in lesdagen_per_jaar.items():
        jaaromzet = Money(0, EUR)
        dagomzet = Money(0, EUR)
        aantal_lesdagen = len(lesdagen_per_jaar[jaar])
        if jaar in row.index:  # is er een gegeven omzet voor dit jaar?
            jaaromzet = Money(row[jaar], EUR)
            dagomzet = jaaromzet / aantal_lesdagen  # wat is de dagomzet voor dit jaar?
            restomzet -= jaaromzet
            debug(f"# Berekening voor {jaar}")
            debug(f"Jaar {jaar} heeft {dagen[0]} als eerste datum.")
            debug(f"Jaar {jaar} heeft {dagen[-1]} als laatste datum.")
            debug(
                f"Jaar {jaar} heeft {jaaromzet} als jaaromzet met dagomzet {dagomzet} in {aantal_lesdagen} werkdagen."
            )
            debug(f"Jaar {jaar} eindigt met restomzet {restomzet}")
        else:  # er is geen gegeven omzet voor dit jaar
            dagomzet = restomzet / aantal_lesdagen  # wat is de dagomzet voor dit jaar?
            debug(
                f"Geen volgende jaaromzet gevonden. Resterende omzet is {restomzet} in {aantal_lesdagen} werkdagen. Dagomzet is {dagomzet}"
            )

        for dag in dagen:
            newrow = {
                "Jaar": dag.year,
                "Maand": pd.to_datetime(f"{dag.year}-{dag.month}-1"),
                "Dag": dag,
                "Omzet": dagomzet,
            }
            df_result = pd.concat(
                [df_result, pd.DataFrame.from_dict([newrow])], ignore_index=True
            )
    df_result.insert(0, "Cursuscode", row["Cursuscode"])

    # elementaire controles
    input_totale_omzet = Money(row["Omzet"], EUR)
    result_totale_omzet = df_result["Omzet"].sum()
    debug("# Controle op gelijkheid omzet...")
    debug(
        f"in: {input_totale_omzet.amount} ({type(input_totale_omzet)}), uit: {result_totale_omzet.amount} ({type(result_totale_omzet)})"
    )
    controle_decimalen = 5
    assert round(result_totale_omzet.amount, controle_decimalen) == round(
        input_totale_omzet.amount, 5
    )  # omzet: som output gelijk aan input?
    debug(
        f"Omzetberekening voor {row['Cursuscode']} correct tot {controle_decimalen} posities achter de komma."
    )

    return df_result

## Een voorbeeldberekening
De testdata is wijzigbaar, probeer dus verschillende dingen uit!

In [37]:
# voorbeelddata
testdata = [
    {
        "Cursuscode": "KP22A",
        "Startdatum": pd.to_datetime("2022-04-01"),
        "Einddatum": pd.to_datetime("2026-02-15"),
        "Omzet": 500_000,
        2022: 100_000,
        2023: 150_000,
    }
]
df = pd.DataFrame(testdata)

result = calc_omzet_per_dag(df.iloc[0, :])

############
Start berekening voor KP22A
# get_vakantiedagen() debuginfo
Eerste vakantiedag: 2019-10-20
Laatste vakantiedag: 2026-08-29
Periode tussen start en einde cursus wordt volledig gedekt door definities vakantieperiodes.
# Restomzet bij aanvang van de berekening is €500,000.00
# Berekening voor 2022
Jaar 2022 heeft 2022-04-01 00:00:00 als eerste datum.
Jaar 2022 heeft 2022-12-23 00:00:00 als laatste datum.
Jaar 2022 heeft €100,000.00 als jaaromzet met dagomzet €662.25 in 151 werkdagen.
Jaar 2022 eindigt met restomzet €400,000.00
# Berekening voor 2023
Jaar 2023 heeft 2023-01-09 00:00:00 als eerste datum.
Jaar 2023 heeft 2023-12-22 00:00:00 als laatste datum.
Jaar 2023 heeft €150,000.00 als jaaromzet met dagomzet €731.71 in 205 werkdagen.
Jaar 2023 eindigt met restomzet €250,000.00
Geen volgende jaaromzet gevonden. Resterende omzet is €250,000.00 in 440 werkdagen. Dagomzet is €568.18
# Controle op gelijkheid omzet...
in: 500000 (<class 'moneyed.classes.Money'>), uit: 499999.9999

## Resultaat
Het resultaat van de voorbeeldbereking in transpositie, zodat de opeenvolgende dagen in kolommen staan.

In [38]:
result.set_index("Dag")[["Omzet"]].transpose()

Dag,2022-04-01,2022-04-04,2022-04-05,2022-04-06,2022-04-07,2022-04-08,2022-04-11,2022-04-12,2022-04-13,2022-04-14,...,2026-02-02,2026-02-03,2026-02-04,2026-02-05,2026-02-06,2026-02-09,2026-02-10,2026-02-11,2026-02-12,2026-02-13
Omzet,€662.25,€662.25,€662.25,€662.25,€662.25,€662.25,€662.25,€662.25,€662.25,€662.25,...,€568.18,€568.18,€568.18,€568.18,€568.18,€568.18,€568.18,€568.18,€568.18,€568.18


Het resultaat van de voorbeeldberekening in transpositie, verdicht naar maanden.

In [39]:
result[["Maand", "Omzet"]].groupby(["Maand"]).sum().transpose()

Maand,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,...,2025-04-01,2025-05-01,2025-06-01,2025-07-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01,2026-01-01,2026-02-01
Omzet,"€13,907.28","€11,258.28","€14,569.54","€3,973.51","€5,298.01","€14,569.54","€10,596.03","€14,569.54","€11,258.28","€12,439.02",...,"€10,795.45","€11,363.64","€11,931.82","€7,954.55","€12,500.00","€10,227.27","€11,363.64","€8,522.73","€11,363.64","€5,681.82"


Het resultaat van de voorbeeldberekening in transpositie, verdicht naar jaren.

In [40]:
result[["Jaar", "Omzet"]].groupby("Jaar").sum().transpose()

Jaar,2022,2023,2024,2025,2026
Omzet,"€100,000.00","€150,000.00","€116,477.27","€116,477.27","€17,045.45"
